<a href="https://colab.research.google.com/github/Niilop/Seq2Seq-text-autofill/blob/main/seq2seq_autofill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data, datasets

import spacy
import numpy as np

import random
import math
import time

Random seed for deterministic results i needed

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

We use Spacy tokenizer for this project

In [3]:
!python3 -m spacy download en_core_web_sm

2023-12-30 19:12:17.840348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 19:12:17.840406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 19:12:17.841560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 19:12:18.861902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Downloading Amazon product dataset from Kaggle

In [4]:
!pip install kaggle

Upload Kaggle API token

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"niilopkknen","key":"b5d82ed4550ac7c2a5b6a897bb44fb17"}'}

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Download required data using Kaggle API

In [8]:
!mkdir amazon_data
!kaggle datasets download -d lokeshparab/amazon-products-dataset
!unzip /content/amazon-products-dataset.zip -d /content/amazon_data

 95% 76.0M/79.7M [00:02<00:00, 42.8MB/s]
100% 79.7M/79.7M [00:02<00:00, 33.1MB/s]
Archive:  /content/amazon-products-dataset.zip
  inflating: /content/amazon_data/Air Conditioners.csv  
  inflating: /content/amazon_data/All Appliances.csv  
  inflating: /content/amazon_data/All Books.csv  
  inflating: /content/amazon_data/All Car and Motorbike Products.csv  
  inflating: /content/amazon_data/All Electronics.csv  
  inflating: /content/amazon_data/All English.csv  
  inflating: /content/amazon_data/All Exercise and Fitness.csv  
  inflating: /content/amazon_data/All Grocery and Gourmet Foods.csv  
  inflating: /content/amazon_data/All Hindi.csv  
  inflating: /content/amazon_data/All Home and Kitchen.csv  
  inflating: /content/amazon_data/All Movies and TV Shows.csv  
  inflating: /content/amazon_data/All Music.csv  
  inflating: /content/amazon_data/All Pet Supplies.csv  
  inflating: /content/amazon_data/All Sports Fitness and Outdoors.csv  
  inflating: /content/amazon_data/All Vid

We will choose "All Electronics" data for now and it contains 8000 unique general electronic products.

In [9]:
import os
current_directory = os.getcwd()
print(current_directory)

/content


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

electronics_df = pd.read_csv('/content/amazon_data/All Electronics.csv')

In [11]:
electronics_df

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999"
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71AvQd3Vzq...,https://www.amazon.in/OnePlus-Nord-Lite-128GB-...,4.3,"113,956","₹18,999","₹19,999"
2,OnePlus Bullets Z2 Bluetooth Wireless in Ear E...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/51UhwaQXCp...,https://www.amazon.in/Oneplus-Bluetooth-Wirele...,4.2,"90,304","₹1,999","₹2,299"
3,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81I3w4J6yj...,https://www.amazon.in/Samsung-Mystique-Storage...,4.1,"24,863","₹15,999","₹24,999"
4,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71V--WZVUI...,https://www.amazon.in/OnePlus-Nord-Black-128GB...,4.3,"113,956","₹18,999","₹19,999"
...,...,...,...,...,...,...,...,...,...
9595,PALAY® BTS Backpack For Boys Kpop BTS Bangtan ...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PALAY%C2%AE-Backpack-Ban...,4.1,37,"₹1,710","₹2,491"
9596,MOCA iPad Air 5th Generation Case 2022 / iPad ...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/MOCA-Protective-Generati...,4.2,"1,133",₹475,"₹1,999"
9597,ZEBRONICS Zeb-Sound Bomb 5 TWS V5.0 Bluetooth ...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Zebronics-Zeb-Sound-Blue...,3.5,"3,177",₹962,"₹3,999"
9598,LINQS® Original NXP Chip | Waterproof NFC Tag ...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/LINQS-Sticker-Smartrac-C...,3.6,210,₹400,"₹1,000"


We only want name of the products

In [12]:
electronics_df['name']

0       Redmi 10 Power (Power Black, 8GB RAM, 128GB St...
1       OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...
2       OnePlus Bullets Z2 Bluetooth Wireless in Ear E...
3       Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...
4       OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...
                              ...                        
9595    PALAY® BTS Backpack For Boys Kpop BTS Bangtan ...
9596    MOCA iPad Air 5th Generation Case 2022 / iPad ...
9597    ZEBRONICS Zeb-Sound Bomb 5 TWS V5.0 Bluetooth ...
9598    LINQS® Original NXP Chip | Waterproof NFC Tag ...
9599    Pragati Systems® Medium Grain 24x36 Inch 10 Oz...
Name: name, Length: 9600, dtype: object

Convert all names to an array

In [13]:
products = electronics_df['name']

In [14]:
spacy_en = spacy.load('en_core_web_sm')

Tokenizer function

In [15]:
def tokeniser(text):
  return[tok.text for tok in spacy_en.tokenizer(text)]

In [16]:
import re
import string
from nltk.corpus import stopwords

def normalize_df(data):
  # Lowercasing
  data = data.str.lower()
  # Removing special characters and punctuation
  data = data.apply(lambda x: re.sub(r'[^\w\s]', '', x))
  return data

def normalize_input(text):

  text = text.lower()

  normalized_text = re.sub(r'[^\w\s]', '', text)

  return normalized_text

In [17]:
processed_data = normalize_df(products)

In [18]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data):
  for sentence in data:
    yield tokeniser(sentence)

vocab = build_vocab_from_iterator(yield_tokens(processed_data), specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=2)
vocab.set_default_index(vocab["<unk>"])


In [19]:
print("The length of the vocab is", len(vocab))
print(f"The index of '<unk>' is {vocab.get_stoi()['<unk>']}")
print(vocab.get_itos())

The length of the vocab is 6552
The index of '<unk>' is 0
['<unk>', '<pad>', '<sos>', '<eos>', ' ', 'for', 'with', 'cover', 'case', 'usb', 'and', 'black', 'gb', 'compatible', 'back', 'to', 'wireless', 'bluetooth', 'pro', '5', 'cable', 'laptop', '2', 'g', 'iphone', 'in', 'samsung', 'smart', 'mm', 'c', 'charging', 'galaxy', '4', 'watch', 'inch', '1', 'ear', 'charger', 'camera', 'adapter', 'fast', '3', 'screen', 'of', 'mic', 'mobile', 'stand', 'holder', 'type', 'ultra', 'pack', 'hd', 'cm', 'phone', 'protection', 'protector', 'portable', 'glass', 'display', '6', 'dual', '8', 'tempered', 'blue', 'power', 'led', 'ram', 'battery', 'pen', 'oneplus', 'earbuds', 'soft', 'edge', 'remote', 'storage', '10', '14', 'gaming', '7', 'full', 'silicone', 'tv', '12', 'clear', 'mouse', 'wired', '360', 'apple', 'band', 'boat', 'tpu', '13', 'plus', 'mi', 'redmi', 'audio', 'mini', 'all', 'bag', 'ipad', '30', 'playtime', 'thegiftkart', 'table', 'design', 'upto', 'speaker', 'gen', 'amazon', 'hard', 'noise', 'sho

Generate data set

We want to split all sentences to pairs in such way:

oneplus nord ce 2 lite 5g black dusk 6gb ram 128gb storage

'&lt;sos&gt; oneplus nord &lt;eos&gt;' , '&lt;sos&gt; ce 2 lite 5g black dusk 6gb ram 128gb storage &lt;eos&gt;'

'&lt;sos&gt; oneplus nord ce &lt;eos&gt;' , '&lt;sos&gt; 2 lite 5g black dusk 6gb ram 128gb storage &lt;eos&gt;'

In [20]:
max_words = 20

def generate_dataset (data):
  pairs = []
  SRC = []
  TRG = []
  sentences = []

  for text in data:

    sentences.append(text)
    words = text.split(' ')

    for i in range(1, len(words) - 1):

        input_data = ' '.join(words[:i+1])
        output_data = ' '.join(words[i+1:])

        #SRC.append(input_data)
        #TRG.append(output_data)
        data = (input_data, output_data)
        pairs.append(data)

  return pairs, sentences

In [21]:
test = processed_data[[4]]
test

4    oneplus nord ce 2 lite 5g black dusk 6gb ram 1...
Name: name, dtype: object

In [22]:
example = generate_dataset(test)
example

([('oneplus nord', 'ce 2 lite 5g black dusk 6gb ram 128gb storage'),
  ('oneplus nord ce', '2 lite 5g black dusk 6gb ram 128gb storage'),
  ('oneplus nord ce 2', 'lite 5g black dusk 6gb ram 128gb storage'),
  ('oneplus nord ce 2 lite', '5g black dusk 6gb ram 128gb storage'),
  ('oneplus nord ce 2 lite 5g', 'black dusk 6gb ram 128gb storage'),
  ('oneplus nord ce 2 lite 5g black', 'dusk 6gb ram 128gb storage'),
  ('oneplus nord ce 2 lite 5g black dusk', '6gb ram 128gb storage'),
  ('oneplus nord ce 2 lite 5g black dusk 6gb', 'ram 128gb storage'),
  ('oneplus nord ce 2 lite 5g black dusk 6gb ram', '128gb storage'),
  ('oneplus nord ce 2 lite 5g black dusk 6gb ram 128gb', 'storage')],
 ['oneplus nord ce 2 lite 5g black dusk 6gb ram 128gb storage'])

Generating pairs from the whole dataset

In [23]:
pairs, sentence = generate_dataset(processed_data)

Process he data. We turn all words in to tokens and change the data type to torch tensors for the PyTorch neural network.

In [24]:
def data_process(pairs):
  data = []
  for (src, trg) in pairs:

    src_tensor = torch.tensor([vocab[token] for token in tokeniser(src)],
                            dtype=torch.long)

    trg_tensor = torch.tensor([vocab[token] for token in tokeniser(trg)],
                            dtype=torch.long)
    data.append((src_tensor, trg_tensor))
  return data

full_data = data_process(pairs)
print(len(full_data))

def input_process(input):

  input_tensor = torch.tensor([vocab[token] for token in tokeniser(input)],
                              dtype=torch.long)
  return input_tensor

full_data = data_process(pairs)

148690


Tokenize the set of pairs to source and target data in a way that the neural network can use

In [25]:
print(full_data[0])

for sentence in full_data[0]:
  words = [vocab.get_itos()[token] for token in sentence]
  print(words)

(tensor([94, 75]), tensor([ 64,  64,  11,  61,  12,  66, 123,  12,  74]))
['redmi', '10']
['power', 'power', 'black', '8', 'gb', 'ram', '128', 'gb', 'storage']


Data Loader

We generate batches for training using a DataLoader and add sos and eos tokens to the sentences.

In [55]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, random_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128
PAD_IDX = vocab['<pad>']
SOS_IDX = vocab['<sos>']
EOS_IDX = vocab['<eos>']

def generate_batch(data_batch):
  src_batch, trg_batch = [], []

  for (src_item, trg_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([SOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
    trg_batch.append(torch.cat([torch.tensor([SOS_IDX]), trg_item, torch.tensor([EOS_IDX])], dim=0))

  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
  trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)
  return src_batch, trg_batch

def generate_input_sequence(input_data):
  input_data = torch.cat([torch.tensor([SOS_IDX]), input_data, torch.tensor([EOS_IDX])]).to(device)
  return input_data

# Split the data to train, eval and test

size = len(full_data)
train_size = int(0.9 * size)
test_eval_size = (size - train_size) // 2

train_data, test_data, val_data = random_split(full_data, [train_size+1, test_eval_size, test_eval_size])


train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=generate_batch)

test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=generate_batch)

val_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=generate_batch)

## Encoder

In [56]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        # Embedding layer
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # GRU
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        # Fully connected layer
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: tensor) -> Tuple[tensor]:

        embedded = self.dropout(self.embedding(src))

        # Forward pass through GRU
        outputs, hidden = self.rnn(embedded)

        hidden_concat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        hidden = torch.tanh(self.fc(hidden_concat))

        return outputs, hidden

## Attention

In [57]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: tensor,
                encoder_outputs: tensor) -> tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        a_score = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(a_score, dim=2)

        # This gives us the attention over the source sentence
        return F.softmax(attention, dim=1)

## Decoder

In [58]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: tensor,
                              encoder_outputs: tensor) -> tensor:

        a = self.attention(decoder_hidden, encoder_outputs)


        a = a.unsqueeze(1)


        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: tensor,
                decoder_hidden: tensor,
                encoder_outputs: tensor) -> Tuple[tensor]:

        #input = [1, batch size]
        input = input.unsqueeze(0)


        #embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)



## Seq2Seq model

In [59]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: tensor,
                trg: tensor,
                teacher_forcing_ratio: float = 0.5) -> tensor:

        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(output, hidden, encoder_outputs)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.max(1)[1]

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            output = (trg[t] if teacher_force else top1)

        return outputs

    def predict(self, src: tensor) -> tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]

        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output

            output = output.argmax(1)  # Greedily select the token with the highest probability

        return outputs

In [60]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = INPUT_DIM

# > 12 mins | Test Loss: 3.063 | Test Perplexity:  21.394 |
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ATTN_DIM = 32
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


# Initialise weights
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01) # Initialise weights with values drawn from the normal distribution
        else:
            nn.init.constant_(param.data, 0) # Initialiase biases to 0


print(model.apply(init_weights))

optimizer = optim.Adam(model.parameters())

# Determine number of paramters
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

# Loss function

PAD_IDX = vocab.get_stoi()['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6552, 128)
    (rnn): GRU(128, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=32, bias=True)
    )
    (embedding): Embedding(6552, 128)
    (rnn): GRU(640, 256)
    (out): Linear(in_features=896, out_features=6552, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)
The model has 8,992,952 trainable parameters


## Train and evaluate functions

In [61]:
import time

def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

##Training

In [ ]:
import math

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 's2s-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Perplexity: {math.exp(train_loss):7.3f} |')
    print(f'\t Val. Loss: {valid_loss:.3f} |   Val Perplexity: {math.exp(valid_loss):7.3f} |')

Epoch: 01 | Time: 3m 0s
	Train Loss: 5.221 | Train Perplexity: 185.152 |
	 Val. Loss: 4.978 |   Val Perplexity: 145.255 |
Epoch: 02 | Time: 2m 59s
	Train Loss: 3.750 | Train Perplexity:  42.524 |
	 Val. Loss: 4.250 |   Val Perplexity:  70.079 |


Loading model weights and evaluating

In [ ]:
# Load the saved model weights
#model.load_state_dict(torch.load('s2s-model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Perplexity: {math.exp(test_loss):7.3f} |')

In [ ]:
src = test_data[2][0].reshape(len(test_data[2][0]),1).to(device)
trg = test_data[2][1].reshape(len(test_data[2][1]),1).to(device)

src_words = [vocab.get_itos()[token] for token in src]
print(src_words)

trg_words = [vocab.get_itos()[token] for token in trg]
print(trg_words)

#print(src)
#print(trg)

In [ ]:
output=model(src, trg)

output_idx = output[1:].squeeze(1).argmax(1)

print(output_idx)

pred_words = [vocab.get_itos()[token] for token in output_idx]
print(pred_words)

In [ ]:
test = ["Samsung"]


In [ ]:
def process_input():
  pass

In [ ]:
def predict_sequence(input, model):
  for data in input:

    normalized_input = normalize_input(data)
    processed_input = input_process(normalized_input)
    input_seq = generate_input_sequence(processed_input)
    input_seq = input_seq.unsqueeze(1)

    model.eval()
    with torch.no_grad():
      output = model.predict(input_seq)

    output_idx = output[1:].squeeze(1).argmax(1)

    topk_values, topk_indices = output[1:].squeeze(1).topk(k=10, dim=1)

    print(f"TOP 10 PREDICTIONS FOR SEQUENCE: {data}")
    for row in topk_indices.t():

      pred_words = [vocab.get_itos()[token] for token in row]
      words_str = data + ' ' + ' '.join(pred_words)
      print(words_str)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
predict_sequence(test, model)